# SNS Text Data Crawilng

In [3]:
import requests
from selenium import webdriver
import time as time
import getpass
import urllib.request
 
from time import sleep
 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def insta_searching(word):
    url = "https://www.instagram.com/explore/tags/" + word
    return url

options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])

## headless setting  /  접속하면 user정보가 없어 크롤링을 막기 때문에 가짜 user-agent 등록
options.headless = True
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.85 Safari/537.36")

## GPU 사용X
# options.add_argument("disable-gpu")

## 언어 = 한글
options.add_argument('lang=ko_KR')


driver = webdriver.Chrome(r"C:\tools\Selenium\chromedriver.exe", options=options) # Chromedriver PATH
driver.get("https://www.instagram.com/accounts/login/")
# driver.maximize_window()
print('드라이버 구동 완료')

# username = getpass.getpass("Input ID : ")      # User ID
username = '01022606785'              

# password = getpass.getpass("Input PWD : ")  # User PWD,  getpass를 통해서 비밀번호 정보를 숨길 수도 있다.
password = 'tr7cfglku12e!@'   


hashTag = input("Input HashTag # : ")# Search #
print("========== 로그인 정보 입력 완료 ==========")

element_id = driver.find_element_by_name("username")
element_id.send_keys(username)         # 개발시 별도의 로그인 계정 확보 필
element_password = driver.find_element_by_name("password")
element_password.send_keys(password)        # 개발시 별도의 로그인 계정 확보 필    
sleep(2)

##로그인버튼 클릭
driver.find_element_by_css_selector('.sqdOP.L3NKy.y3zKF').click()
sleep(2)

## 로그인 정보 저장 관련 
#승인
# driver.find_element_by_css_selector('.sqdOP.L3NKy.y3zKF').click()
#거부
driver.find_element_by_css_selector('.sqdOP.yWX7d.y3zKF').click()

## 검색할 해쉬의 주소
driver.get(insta_searching(hashTag))  
sleep(2)

print('==================검색 완료=================')


## 데이터를 저장할 Dictionary
insta_dict = {'id':[],
              'date': [],
              'text': [],
              'hashtag':[]}
 
 
## 첫 번째 게시물 클릭 
first_post = driver.find_element_by_class_name('eLAPa')
first_post.click()
sleep(3)


seq = 0
start = time.time()
 
while True:
    try:
        if driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow'):
            if seq % 20 == 0:
                print('{}번째 수집 중 / 진행시간 :'.format(seq), time.time() - start, sep = '\t')
  
            ## id 정보 수집
            try:
                info_id = driver.find_element_by_css_selector('h2._6lAjh').text
                insta_dict['id'].append(info_id)
            except:
                info_id = driver.find_element_by_css_selector('div.C4VMK').text.split()[0]
                insta_dict['id'].append(info_id)
 
            ## 시간정보 수집 
            time_raw = driver.find_element_by_css_selector('time.FH9sR.Nzb55')
            time_info = pd.to_datetime(time_raw.get_attribute('datetime')).normalize()
            insta_dict['date'].append(time_info)
 
            ##text 정보수집
            raw_info = driver.find_element_by_css_selector('div.C4VMK').text.split()
            text = []
            for i in range(len(raw_info)):
                ## 첫번째 text는 아이디니까 제외 
                if i == 0:
                    pass
                ## 두번째부터 시작
                else:
                    if '#' in raw_info[i]:
                        pass
                    else:
                        text.append(raw_info[i])
            clean_text = ' '.join(text)
            insta_dict['text'].append(clean_text)
 
            ##hashtag 수집
            raw_tags = driver.find_elements_by_css_selector('a.xil3i')
            hash_tag = []
            for i in range(len(raw_tags)):
                if raw_tags[i].text == '':
                    pass
                else:
                    hash_tag.append(raw_tags[i].text)
 
            insta_dict['hashtag'].append(hash_tag)
 
            seq += 1
            if seq == 100:
                break
 
            driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
            time.sleep(1.5)
        else:
            break
    except:
        print("ID or PW error")
        driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
        time.sleep(2)

print("=============", len(insta_dict['text']), '건의 자료를 수집했습니다.')

드라이버 구동 완료
Input HashTag # : 미나리
========== 로그인 정보 입력 완료 ==========
==================검색 완료=================
0번째 수집 중 / 진행시간 :	0.008155584335327148
20번째 수집 중 / 진행시간 :	34.8642475605011
40번째 수집 중 / 진행시간 :	69.90566039085388
60번째 수집 중 / 진행시간 :	104.68709635734558
80번째 수집 중 / 진행시간 :	139.97725200653076
============= 100 건의 자료를 수집했습니다.


# Crawilng Data → DataFrame

In [4]:
total_data = pd.DataFrame(insta_dict)
total_data

,id,date,text,hashtag
0,with___bb,2021-05-07 00:00:00+00:00,. . 함께 영화보기를 시작한 가족들은 둘찌 처찌 아빠순으로 곯아떨어지고 쌉쌀한 와...,[#미나리]
1,wikitree\n인증됨,2021-05-10 00:00:00+00:00,인증됨 1947년생으로 올해 75세인 윤여정 . . 2주,"[#윤여정, #배우, #미나리, #오스카, #아카데미, #여우조연상, #귀국, #공..."
2,minaobsessed,2021-05-16 00:00:00+00:00,". Her voice, plus that suit. It’s too much 😩♥️ 1주","[#미나리, #트와이스, #Mina, #Minari, #MyouiMina, #Min..."
3,insight.co.kr\n인증됨,2021-05-11 00:00:00+00:00,인증됨 '미나리' 홀대해 인종차별 논란 휩싸인 '골든글로브' 트로피 3개 반납한 톰...,"[#톰크루즈, #골든글로브, #트로피, #보이콧, #골든글로브시상식, #미나리]"
4,sungil_hwang,2021-04-29 00:00:00+00:00,"""우리는 각자 다른 역할을 연기했고, 서로 경쟁 상대가 될 수 없다""며 ""내가 운이...","[#윤며들다, #CNN, #워싱턴포스트, #시사잡지애틀랜틱, #윤여정, #yuhju..."
...,...,...,...,...
95,smkys_100400,2021-05-24 00:00:00+00:00,오늘 디너는 동생친구가 만들어준 미나리아귀찜^^ 17시간,"[#SM미디어, #에스엠미디어, #강윤석, #아귀찜, #미나리]"
96,hemming.96,2021-05-24 00:00:00+00:00,18시간,[#미나리]
97,lovely_winona,2021-05-24 00:00:00+00:00,Yeojung&Kyesang 18시간,"[#ユンヨジョン, #윤여정, #yoonyeojung, #韓国女優, #koreanac..."
98,pila_ment7,2021-05-24 00:00:00+00:00,2021.05.24 18시간,"[#밥상, #밥스타그램, #밥상스타그램, #지..."


# check-point

웹 크롤링 완료 후 네이버 영화 리뷰 데이터를 불러와 데이터 전처리에 집중하였고, 어떤 기술이 있는지? 성능은 어떠한지? 파악하는데 시간을 사용했습니다.<br>
하지만 현재 어느 정도 산출물을 기대하고 있는 시점에서 크롤링한 데이터의 라벨링 작업에 미흡했다는 것이 확인되었습니다.
또한 데이터 라벨링 작업 역시 상당한 시간이 필요한 작업이며, 이를 위해 1. 모델을 개발 2. 긍정, 부정 단어셋 등 원하는 결과를 위한 단어 세트를 구성하는 등의 추가적인 작업이 필요하다는 것을 파악했습니다.<br>
**현재는 해당 내용에 대한 개발이 진행되지 않은 상태**

# 정규표현식 적용

In [13]:
import pandas as pd

# 1. 정규표현식 적용
total_data["text"] = total_data['text'].str.replace("[^0-9^ㄱ-ㅎㅏ-ㅣ가-힣 ]","", regex=True).str.replace("[ㄱ-ㅎㅏ-ㅣ]", "", regex=True).str.replace("^", "", regex=False)

In [16]:
total_data.head()

,id,date,text,hashtag
0,with___bb,2021-05-07 00:00:00+00:00,함께 영화보기를 시작한 가족들은 둘찌 처찌 아빠순으로 곯아떨어지고 쌉쌀한 와인이...,[#미나리]
1,wikitree\n인증됨,2021-05-10 00:00:00+00:00,인증됨 1947년생으로 올해 75세인 윤여정 2주,"[#윤여정, #배우, #미나리, #오스카, #아카데미, #여우조연상, #귀국, #공..."
2,minaobsessed,2021-05-16 00:00:00+00:00,1주,"[#미나리, #트와이스, #Mina, #Minari, #MyouiMina, #Min..."
3,insight.co.kr\n인증됨,2021-05-11 00:00:00+00:00,인증됨 미나리 홀대해 인종차별 논란 휩싸인 골든글로브 트로피 3개 반납한 톰 크루즈 1주,"[#톰크루즈, #골든글로브, #트로피, #보이콧, #골든글로브시상식, #미나리]"
4,sungil_hwang,2021-04-29 00:00:00+00:00,우리는 각자 다른 역할을 연기했고 서로 경쟁 상대가 될 수 없다며 내가 운이 더 좋...,"[#윤며들다, #CNN, #워싱턴포스트, #시사잡지애틀랜틱, #윤여정, #yuhju..."


정규표현식 적용에 대하여 파악 완료하였고 원하는 문장으로 설정하는 것이 가능해졌습니다.

# 데이터 전처리

KoNLpy의 라이브러리인 Kkma, Komoran, Okt, Hannanum<br>
Amazon Comprehend (형태소 분리 및 감정 분석)<br>
Google Cloud Natural Language API (형태소 분리 및 감정 분석)<br>
ETRI(한국전자통신연구원) - 한국어 BERT<br>
kakao - Khaiii(자연어처리)<br>
총 8가지의 라이브러리를 적용해보았다. <br>
<br>
테스트 결과
1. 띄어쓰기가 없는 경우
2. 띄어쓰기가 틀린 경우
3. 맞춤법이 틀린 경우
4. 신조어
<br>
<br>
case에 구애받지 않고 완벽한 형태소 분리가 가능한 모델은 없음.<br>
1, 2의 경우 정규표현식 및 맞춤법검사기, okt()내장 정규화 기능을 활용해 일정 수준의 정규화는 가능하지만, 다수의 오류 발견<br>
3, 4의 경우 spell-checker, 맞춤법 검사 api 혹은 selenium을 활용해 어느 정도 수정이 가능하지만, 맞춤법 검사 기능 역시 다수의 오류가 발견<br>
맞춤법 검사를 지원해주는 네이버 맞춤법검사기와 부산대학교 인공지능연구실과 (주)나라인포테크가 개발한 맞춤법 검사기를 도입해 전처리 테스트를 진행하였으나 역시 완벽한 문장으로 변환해주는 checker는 없음.<br>

<br>
위에 나와있는 자연어 형태소 분류기 및 자연어 처리 라이브러리들을 모두 사용해 보고 비교하였으나,<br>
결론적으로는 자체적으로 커스텀이 필요하다는 결론에 도달하였습니다. <br>
추가적으로 한국어 문장에 대하여 깊이있는 이해와 공부가 필요하다는 결론에 이르렀다. 팀 단위의의 프로젝트를 통해 커스텀된 기술(soynlp)도 발견하였으나, 이를 활용하여 테스트를 진행해보진 못했고 소스코드가 공개되어 있어 이를 살펴보는 중입니다.